In [45]:
from datetime import datetime

import geopandas
import pandas as pd
import requests
import numpy as np

from helpers import *

In [46]:
data = geopandas.read_file("EFSM20_CF_PLD.geojson")
data = data[data.idsource.str.startswith("NAF")]  # north anatolian fault

In [47]:
data

,idfs,idds,idsource,strikemin,strikeavg,strikemax,dipmin,dipavg,dipmax,rakemin,...,mwmaxp02,mwmaxp05,mwmaxavg,mwmaxp95,mwmaxp98,m0rmin,m0rmax,m0ramean,m0rgmean,geometry
419,TRCF045,#16,NAF01,242.0,262.0,299.0,59.0,74.0,90.0,-179.0,...,7.69,7.74,7.96,8.27,8.37,18.6423,18.9648,18.8058,18.7872,"POLYGON ((24.94460 40.21030, 24.94600 40.21100..."
876,TRCF000,#16,NAF03,90.0,108.0,127.0,85.0,87.0,90.0,177.0,...,7.64,7.69,7.91,8.22,8.32,18.7651,18.8582,18.8138,18.8114,"POLYGON ((41.12690 39.31120, 41.12670 39.31120..."
877,TRCF001,#16,NAF04,239.0,244.0,254.0,85.0,87.0,90.0,-178.0,...,7.21,7.26,7.48,7.79,7.89,17.0094,17.4882,17.3110,17.2485,"POLYGON ((34.08910 39.93340, 34.08930 39.93350..."
1120,TRCF046,#16,NAF01,264.0,273.0,289.0,59.0,74.0,90.0,-179.0,...,6.66,6.71,6.93,7.24,7.34,17.3966,17.5093,17.4372,17.4366,"POLYGON ((30.93380 40.79320, 30.93400 40.79330..."
1121,TRCF047,#16,NAF02,250.0,268.0,293.0,74.0,82.0,90.0,-179.0,...,7.64,7.69,7.91,8.22,8.32,18.7887,18.8250,18.8026,18.8025,"POLYGON ((31.25320 40.63830, 31.25420 40.63860..."
1122,TRCF048,#16,NAF02,256.0,274.0,286.0,74.0,82.0,90.0,-179.0,...,6.83,6.88,7.10,7.41,7.51,17.6773,17.7323,17.7009,17.7004,"POLYGON ((30.41700 40.64530, 30.42450 40.66740..."


In [48]:
coordinates = []
for i in range(data.shape[0]):
    coordinates.append(data["geometry"].iloc[i].bounds)

In [49]:
coordinates

[(24.9446, 40.1727, 30.9414, 40.9139),
 (35.9319, 39.3112, 41.1301, 40.5724),
 (34.0891, 39.9273, 35.9319, 40.578),
 (30.9338, 40.7325, 31.5057, 40.8154),
 (31.2532, 40.617, 36.6608, 41.1407),
 (30.417, 40.5743, 31.2635, 40.6674)]

In [50]:
datas = []
for c in coordinates:
    dataurl = f"https://deprem.afad.gov.tr/apiv2/event/filter?limit=1000000&minlat={c[1]}&maxlat={c[3]}&minlon={c[0]}&maxlon={c[2]}&format=json&start=1900-01-01%2000%3A00%3A00&end={datetime.today().isoformat()}&orderby=timedesc"
    data = requests.get(dataurl)
    assert data.status_code == 200
    data = data.json()
    data = pd.DataFrame(data)
    datas.append(data)

datas = pd.concat(datas, ignore_index=True)


In [51]:
datas = datas.astype({
    'rms': np.float64,
    'eventID': str,
    'location': str,
    'latitude': np.float64,
    'longitude': np.float64,
    'depth': np.float64,
    'type': str,
    'magnitude': np.float64,
    'country': str,
    'province': str,
    'district': str,
    'neighborhood': str,
    'date': np.datetime64
})

In [52]:
datas

,rms,eventID,location,latitude,longitude,depth,type,magnitude,country,province,district,neighborhood,date,isEventUpdate,lastUpdateDate
0,0.66,542331,Marmara Denizi - [25.05 km] Silivri (İstanbul),40.841,28.270,7.01,ML,2.0,Türkiye,İstanbul,Silivri,None,2023-01-23 03:53:59.000,False,None
1,0.36,542283,Karacabey (Bursa),40.173,28.270,6.99,ML,1.1,Türkiye,Bursa,Karacabey,Yolağzı,2023-01-22 04:27:03.000,False,None
2,0.26,542179,Erenler (Sakarya),40.651,30.453,7.00,ML,0.9,Türkiye,Sakarya,Erenler,Kanlıçay,2023-01-21 07:35:38.000,False,None
3,0.22,542170,Erenler (Sakarya),40.648,30.469,7.05,ML,1.2,Türkiye,Sakarya,Erenler,Kanlıçay,2023-01-21 06:08:22.000,False,None
4,0.33,542161,Erenler (Sakarya),40.673,30.436,7.03,ML,1.0,Türkiye,Sakarya,Erenler,Tepe,2023-01-21 01:42:35.000,False,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41643,0.00,248770,Akyazı (Sakarya),40.640,30.620,9.00,Md,2.9,Türkiye,Sakarya,Akyazı,Kuzuluk,1999-04-27 21:28:51.480,False,None
41644,0.00,243788,Karapürçek (Sakarya),40.600,30.470,1.00,Md,3.6,Türkiye,Sakarya,Karapürçek,Kanlıçay,1994-05-17 12:38:47.310,False,None
41645,0.90,236231,Merkez (Düzce),40.650,31.260,17.00,Md,2.8,Türkiye,Düzce,Merkez,Yanık,1991-07-01 18:53:42.700,False,None
41646,0.30,236130,Mudurnu (Bolu),40.600,31.260,34.00,Md,2.3,Türkiye,Bolu,Mudurnu,Dereceören,1991-04-27 23:57:31.750,False,None


In [53]:
# Check updated events
datas.eventID.value_counts()

247745    2
281246    2
292543    2
443285    2
292011    2
         ..
74710     1
74686     1
74541     1
74533     1
236130    1
Name: eventID, Length: 41228, dtype: int64

In [54]:
# Sort by time, take the last
datas.sort_values(by="date", inplace=True)
datas.drop_duplicates(subset="eventID", keep="last", inplace=True)

In [55]:
# We need to convert all magnitudes to same types
datas.type.value_counts()

ML     22468
Md     13452
Ml      4895
Mw       397
MW        15
Mwp        1
Name: type, dtype: int64

In [56]:
# Make them lowercase 
datas.type = datas.type.str.lower()
datas.type.value_counts()

ml     27363
md     13452
mw       412
mwp        1
Name: type, dtype: int64

In [57]:
# I don't have a conversion formula for mwp
# Omit that obs
datas = datas[datas.type != 'mwp']
datas.shape

(41227, 15)

In [58]:
# Apply conversation formulas
# Source: https://aj.tubitak.gov.tr/earth/issues/yer-16-25-4/yer-25-4-2-1511-7.pdf
datas['magnitude'] = datas[['magnitude', 'type']].apply(convertif, axis=1)

C:\Users\murat\AppData\Local\Temp\ipykernel_5148\295769714.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datas['magnitude'] = datas[['magnitude', 'type']].apply(convertif, axis=1)


In [59]:
# All magnitudes are in Mw now
# Drop columns
cols = ['type', 'neighborhood', 'isEventUpdate', 'lastUpdateDate']
datas.drop(labels=cols, axis=1, inplace=True)
datas

C:\Users\murat\AppData\Local\Temp\ipykernel_5148\1492905530.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datas.drop(labels=cols, axis=1, inplace=True)


,rms,eventID,location,latitude,longitude,depth,magnitude,country,province,district,date
41476,0.00,237968,Mengen (Bolu),41.000,32.000,7.00,3.72610,Türkiye,Bolu,Mengen,1990-01-06 12:59:39.000
30103,0.00,237972,Merkez (Sivas),39.740,36.960,1.00,3.96451,Türkiye,Sivas,Merkez,1990-01-08 10:44:47.640
16445,0.00,237974,Göynük (Bolu),40.370,30.810,7.00,3.09034,Türkiye,Bolu,Göynük,1990-01-17 15:49:11.630
33121,0.00,237980,Sungurlu (Çorum),40.270,34.480,1.00,3.80557,Türkiye,Çorum,Sungurlu,1990-01-29 15:33:14.000
41475,0.00,237985,Mengen (Bolu),41.000,32.000,4.00,3.72610,Türkiye,Bolu,Mengen,1990-02-02 22:07:34.000
...,...,...,...,...,...,...,...,...,...,...,...
33975,0.59,542319,Tosya (Kastamonu),40.922,33.826,7.00,2.19075,Türkiye,Kastamonu,Tosya,2023-01-23 00:01:21.000
16447,0.38,542320,Merkez (Bayburt),40.107,40.492,15.12,2.59550,Türkiye,Bayburt,Merkez,2023-01-23 00:53:19.000
16446,0.69,542327,Otlukbeli (Erzincan),40.043,40.019,7.01,2.27170,Türkiye,Erzincan,Otlukbeli,2023-01-23 02:46:50.000
0,0.66,542331,Marmara Denizi - [25.05 km] Silivri (İstanbul),40.841,28.270,7.01,2.91930,Türkiye,İstanbul,Silivri,2023-01-23 03:53:59.000
